In [1]:
import math
import pandas
import sqlalchemy
print(pandas.__version__)

import geopandas as gpd
import folium
#from folium.plugins import MarkerCluster # for clustering the markers
import plotly.express as px
import plotly.graph_objects as go

# 
pandas.set_option('max_columns', None)

1.3.4


In [8]:
# Path to sql query files
SQL_PATH = r"./sqlCommands/"
SQL_DATA = r"loadMastrSolar.sql"

SQL_DATA_PATH = SQL_PATH + SQL_DATA

# Parameters to create a connection to the MaStR-postgreSQL DB
CONN_PARAM_DICT = {
    "host": "10.0.0.102",
    "dbname": "mastr",
    "user": "uba_user",
    "password": "UBAit2021!",
    "port": "5432"
}

bundesländer = {
    1400:"Brandenburg",
    1401:"Berlin",
    1402:"Baden-Würtenberg",
    1403:"Bayern",
    1404:"Bremen",
    1405:"Hessen",
    1406:"Hamburg",
    1407:"Mecklenburg-Vorpommern",
    1408:"Niedersachsen",
    1409:"Nordrhein-Westfahlen",
    1410:"Rheinland-Pfalz",
    1411:"Schleswig-Holstein",
    1412:"Saarland",
    1413:"Sachsen",
    1414:"Sachsen-Anhalt",
    1415:"Thüringen"}

In [3]:
def build_postgres_conn_string (param:dict) -> str:
    return f'postgresql+psycopg2://{param["user"]}:{param["password"]}@{param["host"]}:{param["port"]}/{param["dbname"]}'

def create_postgres_engine (param:dict) -> sqlalchemy.engine.base.Engine:
    conString = build_postgres_conn_string(param)
    engine = sqlalchemy.create_engine(conString, pool_recycle=3600)
    return engine

def read_postgres_from_queryfile (sqlpath:str, postgresLogin:dict) -> pandas.DataFrame:
    engine = create_postgres_engine(postgresLogin)
    
    scriptFile = open(sqlpath,'r')
    script = scriptFile.read()
    df = pandas.read_sql(script, engine)

    return df

In [9]:
dfMastrSolar = read_postgres_from_queryfile(sqlpath=SQL_DATA_PATH, postgresLogin=CONN_PARAM_DICT)

dfMastrSolar["Anlagenzahl"] = 1

dfMastrSolar = dfMastrSolar.replace({"Bundesland": bundesländer})

dfMastrSolar.columns

Index(['EinheitMastrNummer', 'DatumLetzteAktualisierung',
       'LokationMaStRNummer', 'NetzbetreiberpruefungStatus',
       'NetzbetreiberpruefungDatum', 'AnlagenbetreiberMastrNummer', 'Land',
       'Bundesland', 'Landkreis', 'Gemeinde', 'Gemeindeschluessel',
       'Postleitzahl', 'Ort', 'Registrierungsdatum', 'Inbetriebnahmedatum',
       'EinheitSystemstatus', 'EinheitBetriebsstatus',
       'NichtVorhandenInMigriertenEinheiten', 'NameStromerzeugungseinheit',
       'Weic_nv', 'Kraftwerksnummer_nv', 'Energietraeger', 'Bruttoleistung',
       'Nettonennleistung', 'FernsteuerbarkeitNb', 'Einspeisungsart',
       'ZugeordneteWirkleistungWechselrichter',
       'GemeinsamerWechselrichterMitSpeicher', 'AnzahlModule', 'Lage',
       'Leistungsbegrenzung', 'EinheitlicheAusrichtungUndNeigungswinkel',
       'Hauptausrichtung', 'HauptausrichtungNeigungswinkel', 'Nutzungsbereich',
       'EegMaStRNummer', 'Nebenausrichtung', 'NebenausrichtungNeigungswinkel',
       'Strasse', 'StrasseNicht

In [18]:
datum = dfMastrSolar["Inbetriebnahmedatum"] > "2010"
leistung_bot = dfMastrSolar["Nettonennleistung"] > 400
leistung_top = dfMastrSolar["Nettonennleistung"] < 500

spalten = ['Laengengrad', 'Breitengrad']

dfMastrSolar.loc[leistung_bot & leistung_top & datum,spalten]

,Laengengrad,Breitengrad
416,7.900971,51.985225
1600,9.113325,49.835014
1607,8.627036,53.458855
3041,9.182805,53.976083
6883,11.629569,49.963936
...,...,...
2242904,12.549795,53.961838
2244117,7.187727,52.682175
2246124,7.637206,52.314271
2248470,11.957701,52.443406


In [ ]:
def myfunc(age, pclass):
    if pandas.isnull(age) and pclass==1:
        age=40
    elif pandas.isnull(age) and pclass==2:
        age=30
    elif pandas.isnull(age) and pclass==3:
        age=25
    else:
        age=age
    return age

df['NewColumn_1'] = df.apply(lambda x: myfunc(x['Age'], x['Pclass']), axis=1)

In [28]:
def classify_Leistung(nennLeistung):
    if  nennLeistung<=10:
        klasse=1
    elif nennLeistung>10 and nennLeistung<=100:
        klasse=2
    elif nennLeistung>100 and nennLeistung<=400:
        klasse=3
    else:
        klasse=4
    return klasse

dfMastrSolar['LeistungsKlassen'] = dfMastrSolar.apply(lambda x: classify_Leistung(nennLeistung=x['Nettonennleistung']), axis=1)
dfMastrSolar.loc[:,['Nettonennleistung','LeistungsKlassen']]

,Nettonennleistung,LeistungsKlassen
0,9.000,1
1,3.000,1
2,11.300,2
3,16.200,2
4,7.500,1
...,...,...
2250021,29.970,2
2250022,4.100,1
2250023,12.245,2
2250024,9.440,1


In [11]:
dfMastrSolar2020 = dfMastrSolar[dfMastrSolar["Inbetriebnahmedatum"].dt.year == 2020]

dfMastrSolar2020.groupby(by=["Bundesland"]).sum()[["Anlagenzahl","Nettonennleistung"]]

,Anlagenzahl,Nettonennleistung
Bundesland,,
Baden-Würtenberg,32583,560394.066
Bayern,47132,1212418.273
Berlin,1528,15950.649
Brandenburg,6483,383897.459
Bremen,303,3830.033
Hamburg,643,8262.660
Hessen,11701,201506.869
Mecklenburg-Vorpommern,2508,328593.352
Niedersachsen,17649,373842.565
